In [ ]:
from nbdev import *
%nbdev_default_export cli

Cells will be exported to RemoteConnect.cli,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# cli

> This is a cli module for RemoteConnect

In [ ]:
%nbdev_export
from nbdev.showdoc import *
from fastscript import call_parse, Param, bool_arg
from RemoteConnect.core import RemoteCode, RemoteJupyter

In [ ]:
%nbdev_export
@call_parse
def start_code(port:Param("Port to Start Code", type=int)=10000, 
               password:Param("Password to Start Code", type=str)=None):
    "Starts Code Server"
    print(RemoteCode(password=password, port=port))


In [ ]:
%nbdev_export
@call_parse
def start_jupyter(port:Param("Port to Start Code", type=int)=9000):
    "Starts Jupyter"
    RemoteJupyter(port=port)